In [ ]:
'''
TODO:

PROBLEMS:
1. Наличие освещения -> bool/OHE? Или int, так как есть градация в виде сумерек? 
2. ~5000 строк не включают информацию о водителе -- убрано
3. Схемы - это, конечно, интересно, но что с ними делать?
4. Severity -> int?
5. Время перевести в минуты?
'''

In [1]:
import pandas as pd
import numpy as np
from io import StringIO
from csv import writer
from IPython.core.display import display, HTML

In [2]:
# функция для преобразования кода съемы в HTML-код изображения на сервере сайта dtp-stat.ru

def path_to_image_html(path):
    return '<img src="https://dtp-stat.ru/static/scheme/'+ path + '.png">'

In [3]:
# функция для подготовки One-Hot кодирования столбца, в котором данные имеют вид списков
# возвращает набор присутствующих значений и DataFrame-объект закодированной таблицы 

def one_hot_prepare(datacol):      
    ohe_cols = []
    for value in datacol.values:
        for i in range(len(value)):
            if value[i] not in ohe_cols:
                ohe_cols.append(value[i])
                
    res = dict(zip(ohe_cols, [[None]*len(datacol) for _ in ohe_cols]))
    
    for i, value in enumerate(datacol):
        for col in ohe_cols:
            if col in value:
                res[col][i] = 1
            else:
                res[col][i] = 0      
    return ohe_cols, res
    

In [4]:
# функция для осуществления One-Hot кодирования указанных столбцов в заданной таблице

def one_hot_encoding(data, columns):
    for column in columns:
        ohe_headers, ohe_data = one_hot_prepare(data[column])
        for header in ohe_headers:
           data[header] = ohe_data[header]
        data = data.drop(columns=column)
    return data        

In [5]:
with open("D://spb.geojson", encoding='utf-8') as f:
    data = pd.read_json(f)

In [6]:
# извлекаем нужные данные путём нормализации json'a
# в несколько этапов - для того, чтобы не было лишних префиксов

data = pd.json_normalize(data['features'], sep='_', max_level=0)
data = pd.json_normalize(data['properties'], sep='_')

In [7]:
# удаляем лишние столбцы и фильтруем данные, оставляя связанные с перекрёстками (согласно схемам)

cutted = data.drop(columns=['id', 'gibdd_id', 'participant_categories', 'parent_region', 'address', 'vehicles', 'participants', 'tags'])
scheme_filter = ['100', '110', '120', '130', '300', '310', '320', '340', '400', '410', '420', '430', '440', 
                 '700', '710', '720', '730', '760', '770', '780']
cutted = cutted[cutted.scheme.isin(scheme_filter)]

In [8]:
# визуализируем схемы ДТП

pd.set_option('display.max_colwidth', None)
format_dict = {'scheme': path_to_image_html}
display(HTML(cutted.tail(3).to_html(escape=False ,formatters=format_dict)))

,severity,region,datetime,participants_count,injured_count,dead_count,category,light,nearby,weather,road_conditions,scheme,point_lat,point_long
39445,Тяжелый,Адмиралтейский район,2017-02-13 14:40:00,2,1,0,Наезд на пешехода,Светлое время суток,"[Мост, Регулируемый пешеходный переход]",[Пасмурно],[Мокрое],,59.908300,30.27610
39446,Легкий,Адмиралтейский район,2017-02-24 21:00:00,2,1,0,Наезд на пешехода,"В темное время суток, освещение включено","[Многоквартирные жилые дома, Регулируемый пешеходный переход]",[Пасмурно],[Мокрое],,59.924190,30.30814
39447,Легкий,Адмиралтейский район,2017-02-21 01:00:00,2,1,0,Наезд на пешехода,Светлое время суток,"[Многоквартирные жилые дома, Нерегулируемый пешеходный переход]",[Пасмурно],[Мокрое],,59.914313,30.29749


In [9]:
# преобразуем datetime в месяц, день недели и время

datetime = cutted['datetime']
month = []
day = []
time = []
for row in datetime:
    day.append(pd.to_datetime(row).dayofweek)
    row = row.split(' ')
    month.append(row[0].split('-')[1])
    time.append(row[1])

cutted['month'] = month
cutted['day'] = day
cutted['time'] = time

In [10]:
result = pd.get_dummies(cutted, columns=['severity', 'region', 'category', 'light'], prefix=['Тяжесть', 'Район', 'Категория', 'Свет'])
result = one_hot_encoding(result, ['weather', 'nearby', 'road_conditions'])

In [11]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9388 entries, 0 to 39447
Columns: 139 entries, datetime to Неудовлетворительное состояние разделительной полосы
dtypes: float64(2), int64(98), object(4), uint8(35)
memory usage: 7.8+ MB


In [47]:
# One-Hot для столбца light

# raw_light = cutted['light']
# light = raw_light.unique()

# sun_light = []
# art_light = []

# for i in range(len(light)):
#     light[i] = light[i].split(', ')
#     for j in range(len(light[i])):        
#         if j == 0 and light[i][j] not in sun_light:
#             sun_light.append(light[i][j])        
            
#         elif j == 1 and light[i][j] not in art_light:
#             art_light.append(light[i][j])

# cols = sun_light + art_light
# res = dict(zip(cols, [[None]*len(raw_light) for _ in cols]))

# for i, row in enumerate(raw_light):
#     for col in cols:
#         if col in row:
#             res[col][i] = 1
#         else:
#             res[col][i] = 0
            
# for col in cols:
#         cutted[col] = res[col]
